In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import emcee
from find_source import summary

In [2]:
# logarithmic likelihood function (for 2D Gaussian probability)
def log_likelihood(mu, re, im, w):
    re_mean, im_mean = mu
    return -0.5 * np.sum(w * ((re - re_mean)**2 + (im - im_mean)**2))

In [3]:
def log_prior(mu, re_range, im_range):
    re_mean, im_mean = mu
    if re_range[0] < re_mean < re_range[1] and im_range[0] < im_mean < im_range[1]:
        return 0.0
    return -np.inf
    return 0.0

In [4]:
def log_probability(mu, re_range, im_range, re, im, w):
    lp = log_prior(mu, re_range, im_range)
    if not np.isfinite(lp): # if outside of range where mu could be
        return -np.inf # essentially 0 probability (e^-inf = 0)
    return lp + log_likelihood(mu, re, im, w)

In [5]:
##### FAKE ARRAY IN FORM OF KARTO'S DATA #####
def fake_data():
    fake_arr = np.tile([float(0)]*6, (20, 1))
    for i in range(len(fake_arr)):
        fake_arr[i][1] = i
        fake_arr[i][2] = i
        real = 5e-3 # 5 mJy source
        imag = 0
        x = 1e-3 # 1 mJy standard deviation
        re_noise = np.random.normal(scale = x)
        im_noise = np.random.normal(scale = x)
        fake_arr[i][3] = real + re_noise
        fake_arr[i][4] = imag + im_noise
        weight = (re_noise**2 + im_noise**2)**(-1)
        fake_arr[i][5] = weight
        fake_arr[i][3] *= weight # weighting reals like Karto
        fake_arr[i][4] *= weight # weighting imaginaries like Karto
    return fake_arr

In [6]:
def mcmc_uv_fit(fits_file: str):
    # TODO: generalize to fitting n point/disk/Gaussian sources

    file = fits.open(fits_file)
    data = file[1].data
    vis = np.array(data)
    freq_bin, u, v, re, im, w = [], [], [], [], [], []
    for row in vis:
        freq_bin_data, u_data, v_data, re_data, im_data, w_data = row
        freq_bin.append(int(freq_bin_data))
        u.append(int(u_data))
        v.append(int(v_data))
        re.append(float(re_data/w_data))
        im.append(float(im_data/w_data))
        w.append(float(w_data))

    # adding in conjugate half of data
    freq_bin *= 2
    neg_u = [-1 * val for val in u]
    u += neg_u
    neg_v = [-1 * val for val in v]
    v += neg_v
    re *= 2
    neg_im = [-1 * val for val in im]
    im += neg_im
    w *= 2
    rlambda = []
    for i in range(len(u)):
        rlambda.append((u[i]**2 +v[i]**2)**(1/2))

    freq_bin = np.array(freq_bin)
    u = np.array(u)
    v = np.array(v)
    re = np.array(re)
    im = np.array(im)
    w = np.array(w)
    rlambda = np.array(rlambda)

    # TODO: optional parameters: nwalkers, number of sources (therefore number of dimensions), nsteps, discard number
    nwalkers = 6
    nsteps = 2000 # TODO: automatically stop after ~50x autocorrelation time
    ndim = 2

    # initial positions for re and im
    p0 = np.random.randn(nwalkers, ndim) * np.average(w**(-1/2))
    p0[:,0] += np.average(re)
    p0[:,1] += np.average(im)

    re_range = (-2 * np.max(np.abs(re)), 2 * np.max(np.abs(re)))
    im_range = (-2 * np.max(np.abs(im)), 2 * np.max(np.abs(im)))

    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(re_range, im_range, re, im, w))
    sampler.run_mcmc(p0, nsteps)
    chain = sampler.get_chain(discard = 100)

    return chain

In [7]:
mcmc_uv_fit('../data/uv_test/at2024tvd.fits')

array([[[ 2.94676982e-04, -3.63488674e-05],
        [ 3.27922400e-04, -7.80192647e-05],
        [ 5.84130943e-04, -3.16161084e-05],
        [ 4.41261778e-04,  4.78298102e-05],
        [ 5.48841883e-04, -6.64792405e-05],
        [ 4.00117516e-04, -2.10403080e-04]],

       [[ 2.26175802e-04,  7.67283215e-05],
        [ 3.27922400e-04, -7.80192647e-05],
        [ 6.88801056e-04, -8.98205027e-05],
        [ 3.98303248e-04,  1.29784298e-07],
        [ 5.48841883e-04, -6.64792405e-05],
        [ 4.00117516e-04, -2.10403080e-04]],

       [[ 2.76853616e-04,  5.42361952e-05],
        [ 3.78730636e-04, -7.96807653e-05],
        [ 6.76603383e-04, -8.94216217e-05],
        [ 3.98303248e-04,  1.29784298e-07],
        [ 6.54502201e-04, -1.13374025e-04],
        [ 4.00355147e-04, -2.37978531e-04]],

       ...,

       [[ 3.59904291e-04,  1.40462599e-04],
        [ 6.14222760e-04,  6.55099152e-05],
        [ 4.58860570e-04,  9.84794106e-05],
        [ 6.13745957e-04, -2.08115099e-06],
        [ 5.1